Setup globals

In [1]:
import datetime
import gmaps
import gmaps.datasets
import math
import os.path
import pandas as pd
import sqlite3
import time
from IPython.display import display, Markdown

day_of_seconds = 86400
week_of_seconds = 7 * day_of_seconds
month_of_seconds = 30 * day_of_seconds

today = int(time.mktime(datetime.date.today().timetuple()))

The following cell values are safe to modify as needed

In [2]:
sqlite_file = 'RaMBLE.sqlite'
gmaps_apikey_filename = 'gmaps_api.key'
max_rows = 500
gmaps

# Typically only looking back the past week
wayback = week_of_seconds

Pull in database

In [3]:
pd.set_option('display.max_rows', max_rows)
db = sqlite3.connect(sqlite_file)
device_table = pd.read_sql_query("SELECT * from devices", db)
lastest_activity = device_table.last_seen.max()

new_devices = device_table[device_table.first_seen >= lastest_activity-wayback]
old_devices = device_table[device_table.first_seen < lastest_activity-wayback]

The database has two important tables: `devices` and `locations`

devices
* id (int)
* address (string: OUI)
* adv_flags (float)
* device_name (string)
* device_type (string)
* first_seen (int)
* last_seen (int)
* msd_key (float, Manufacturer Data: ID)
* msd_field (hex string, Manufacturer Data: Data)
* service_uuids (string uuid)
* service_data (string hex data)
* days_seen (int)
* raw_adv_data (bitstring)

locations
* id (int)
* device_id (int)
* timestamp (int)
* rssi (int)
* tx_power (float)
* latitude (float)
* logitude (float)
* accuracy (float)
* packets_received (int)

In [4]:
oui_table = pd.read_csv('data/nmap-mac-prefixes', sep='\t', error_bad_lines=False, names=["oui", "vendor"])
cid_table = pd.read_csv('data/cid.csv',  header=0)
msd_table = pd.read_csv('data/bt_msdid.csv')

def grab_org(address, msd):
    oui_octets = address.split(':')[:3]
    mac_type = 'cid'
    org_name = None
    mfg_name = None
    
    # turn off the multicast bit
    oui_octets[0] = '{:02x}'.format(int(oui_octets[0], 16) & 0xfe)
    oui = ''.join(oui_octets).upper()
    
    # check to see if this is a global oui
    if int(oui_octets[0], 16) & 0x2 == 0:
        mac_type = 'oui'
        org_name = oui_table[oui_table.oui == oui].vendor.values
    else:
        org_name = cid_table[cid_table.Assignment == oui]['Organization Name'].values
        
    if len(org_name) > 0:
        org_name = org_name[0]
    else:
        org_name = mac_type+'/'+':'.join(oui_octets)
        
    if math.isnan(msd) is False:
        mfg_name = msd_table[msd_table.Decimal == int(msd)].Company.values
    
        if len(mfg_name) > 0:
            mfg_name = mfg_name[0]
        else:
            mfg_name = int(msd)

    return org_name, mfg_name

def device_data(device_id):
    data = new_device.loc[device_id, extract_columns].values.tolist()

    org_data = grab_org(data[2], data[5])

    days_seen = data[4]
    data = data[:-2]
    data.extend(org_data)
    data.append(days_seen)
    
    return data

old_names = old_devices['device_name'].dropna(how='any').unique()

# Find devices with device types never seen before
new_names = new_devices.copy()
for old_name in old_names:
    new_names = new_names[new_names.device_name != old_name]

# format count table
name_count = new_names.device_name.value_counts()
name_count.name = ''
name_count = pd.DataFrame(name_count).sort_index()
name_count.index.names = ["device_name"]

extract_columns = ["device_name", "device_type", "address", "service_uuids", "days_seen", "msd_key"]
context_columns = ["device_name", "device_type", "address", "service_uuids", "org name", "manufacturer", "days_seen"]

device_report = list()

# grab contextual data on devices and add them to our device report
for new_name in name_count.index:
    new_device = new_devices[new_devices.device_name == new_name]
    
    # Add devices to our report list
    for device_id in new_device.index:
        device_report.append(device_data(device_id))

# Find devices with device types never seen before

old_types = old_devices['device_type'].dropna(how='any').sort_values().unique()
new_types = new_devices.copy()

for old_type in list(old_types):
    new_types = new_types[new_types.device_type != old_type]

# format count table
type_count = new_types.device_type.value_counts()
type_count.name = ''
type_count = pd.DataFrame(type_count).sort_index()
type_count.index.names = ["device_type"]

# Add any of these devices to our device list if we haven't already

for new_type in type_count.index:
    new_device = new_devices[new_devices.device_type == new_type]
    
    for device_id in new_device.index:
        # Check to see if they would have been reported for their name
        if new_device.loc[device_id, "device_name"] in name_count.index:
            continue
        
        # They haven't been reported before, add them
        device_report.append(device_data(device_id))

# Find devices with Service UUIDs never seen before

old_uuids = old_devices['service_uuids'].dropna(how='any').unique()

new_uuids = new_devices.copy()
for old_uuid in old_uuids:
    new_uuids = new_uuids[new_uuids.service_uuids != old_uuid]

# format count table
uuid_count = new_uuids.service_uuids.value_counts()
uuid_count.name = ''
uuid_count = pd.DataFrame(uuid_count).sort_index()
uuid_count.index.names = ["service_uuids"]

# Add any of these devices to our device list if we haven't already

for new_uuid in uuid_count.index:
    new_device = new_devices[new_devices.service_uuids == new_uuid]
    
    for device_id in new_device.index:
        # Check to see if they would have been reported for their name
        if new_device.loc[device_id, "device_name"] in name_count.index:
            continue
        
        # Check to see if they would have been reported for their type
        if new_device.loc[device_id, "device_type"] in type_count.index:
            continue
        
        # They haven't been reported before, add them
        device_report.append(device_data(device_id))

# Turn our list into a dataframe
if len(device_report) > 0:
    device_report = pd.DataFrame(device_report)
    device_report.columns = context_columns
else:
    device_report = pd.DataFrame(columns = context_columns)
    # Dump report to csv
    device_report.to_csv('interesting_devices.csv')

In [5]:
display(Markdown("# New Devices of Interest"))
display(Markdown(f"{old_devices.id.count()} devices from {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(old_devices.iloc[0].first_seen))} to {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(old_devices.iloc[-1].first_seen))}"))
display(Markdown(f"{new_devices.id.count()} devices from {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(new_devices.iloc[0].first_seen))} to {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(new_devices.iloc[-1].first_seen))}"))

# New Devices of Interest

10545 devices from 2018-11-20 15:12:37 to 2020-03-13 17:39:36

1156 devices from 2020-08-31 18:06:10 to 2020-09-05 13:31:39

In [6]:
name_count

,
device_name,
00A05020BEF01B28z22,1
00A05020E1A50D200T0,1
00A05020E1B20028z02,1
ALAM (24:F7:2C) ,1
BLE-OHL-01,1
BLE_08A0,1
BLE_891B,1
BLE_92EB,1
BLE_C826,1


In [7]:
type_count

,
device_type,
"Apple, Wireless Audio",4
Bose,3
"Bose, SGL Italia",2
CSR,1
CUBE TECHNOLOGIES,3
Device Information,6
"Device Information, Fitbit",1
Eddystone UID beacon,1
Eddystone URL beacon - http://mrefer.com/4153232615,1


In [8]:
uuid_count

,
service_uuids,
0000,1
00000000-0200-a58e-e411-afe28044e62c,1
00001533-1412-efde-1523-785feabcd123,1
3910bf96-4348-f3e4-a002-6afb7fc989ae,1
447c0000-ed49-fe29-71e1-87a4e15ecff6,1
47726f74-6547-4c4f-5353-414253000000,1
6e400001-b5a3-f393-e0a9-e50e24dcca9e,1
88e0,1
fd6f,1


In [9]:
if os.path.exists(gmaps_apikey_filename):
    gmaps_api = ''
    
    with open(gmaps_apikey_filename, 'r') as file:
        gmaps_api = file.read().replace('\n', '')
    
    gmaps.configure(api_key=gmaps_api)
    
    locations = pd.read_sql_query("SELECT * from locations", db)
    stationary_devices = device_report[device_report['days_seen'] > 1]
    print("Devices only seen one day")
    display(device_report[device_report['days_seen'] == 1].sort_values(by="address")[context_columns[:-1]])
    
    print("Devices seen multiple days")
    
    for x in range(len(stationary_devices.index)):
        display(stationary_devices.iloc[[x]])
        
        device_ids = new_devices[new_devices.address == stationary_devices.iloc[[x]].address.values[0]].id
        device_locations = locations[locations['device_id'] == device_ids.values[0]][['latitude', 'longitude']]
        device_rssi = locations[locations['device_id'] == device_ids.values[0]][['rssi']]
        
        #display(device_rssi.dtypes)
        weights = list()
        
        for weight in device_rssi.values:
            weights.append(weight+101)

        fig = gmaps.figure()
        fig.add_layer(gmaps.heatmap_layer(device_locations))
        display(fig)
else:
    print()
    display(Markdown("**Location mapping not enabled** To enable heatmaps of stationary devices, save your Google Maps Javascript SDK API Key in the file `gmaps_api.key`"))
    display(device_report.sort_values(by="address"))

**Location mapping not enabled** To enable heatmaps of stationary devices, save your Google Maps Javascript SDK API Key in the file `gmaps_api.key`

,device_name,device_type,address,service_uuids,org name,manufacturer,days_seen
0,00A05020BEF01B28z22,Battery Service,00:01:AD:66:08:66,180f,Coach Master d.b.a. CMI Worldwide,None,1
2,00A05020E1B20028z02,Battery Service,00:01:AD:71:87:09,180f,Coach Master d.b.a. CMI Worldwide,"Apple, Inc.",1
24,G-519C9C,None,00:05:C2:51:9C:9C,47726f74-6547-4c4f-5353-414253000000,Soronti,None,1
39,Micro Mechanic,None,00:1D:A5:00:F9:57,fff0,WB,None,2
45,PayPal-10395335,None,00:1D:FA:20:09:31,None,Fujian LANDI Commercial Equipment,None,2
36,MSH317_Ble_C7A,CUBE TECHNOLOGIES,00:22:6C:D6:8C:7B,"ffc0,6666",LinkSprite,CUBE TECHNOLOGIES,1
116,None,"Hewlett-Packard Company, HP",00:68:EB:FB:22:E3,fe78,HP,Hewlett-Packard Company,1
1,00A05020E1A50D200T0,Battery Service,00:A0:50:20:E1:A5,180f,Cypress Semiconductor,None,1
118,None,"Hewlett-Packard Company, HP",02:68:EB:45:2C:20,fe78,cid/02:68:EB,Hewlett-Packard Company,2
115,None,"Hewlett-Packard Company, HP",02:68:EB:46:80:9B,fe78,cid/02:68:EB,Hewlett-Packard Company,3
